In [1]:
import numpy as np


import pandas as pd 

# importing functions
import sys
sys.path.append('/Users/audreyburggraf/Desktop/THESIS/Functions')

from functions import *
from unit_conversion_functions import *
from fitting_functions import *
from signal_functions import *
from printing_functions import *

In [2]:
df = pd.read_csv('/Users/audreyburggraf/Desktop/THESIS/Gaia Data Files/gaia_data.csv')

#  add the absolute gaia K band magnitude and stellar mass
df["M_ks"] = df.ks_m + 5 + 5*np.log10(df.parallax/1000)

# cut off certain range 
df = df[(4 < df.M_ks) & (df.M_ks < 11)].reset_index(drop=True)

# set the stellar mass 
df["stellar_mass"] =  find_star_mass(df.M_ks)

In [3]:
N_synthetic = 70
N_model = 70000

np.random.seed(5) 


# call function
inj_params_0P, inj_params_1P, synthetic_signal, model_signal, error_components, alpha, m_star = find_signal_components(df, N_synthetic, N_model)

Planetary Parameters:
e              : 0.11099658554486974  [unitless]
omega          : 5.470972432660289  [radians]
Omega          : 1.2988547595412585  [radians]
cos_i          : 0.9186109079379216  [unitless]
log10(m_planet): 0.27656348124786  [log10(Jupiter masses)]
log10(P_orb)   : -0.16476841129206288  [log10(years)]
t_peri         : 0.5240927031418511  [years]
 
Gaia Parameters:
alpha0         : 177.55505174044077  [degrees]
delta0         : -52.96074908143371  [degrees]
mu_alpha       : -134.1902063718185  [mas/year]
mu_delta       : 16.283442467935146  [mas/year]
parallax       : 16.28087886268879  [mas]
m_star         : 0.6599415904544755  [M_sun]
x              : 199326
 
Astrometric signature: 30.098325405669197 [uas]


In [4]:
# Unpack the result statement

# Unpack synthetic signal components and times
(prop_ra_synthetic, prop_dec_synthetic, 
 parallax_ra_synthetic, parallax_dec_synthetic, 
 planetary_ra_synthetic, planetary_dec_synthetic, 
 times_synthetic) = synthetic_signal

# Unpack model signal components and times
(prop_ra_model, prop_dec_model, 
 parallax_ra_model, parallax_dec_model, 
 planetary_ra_model, planetary_dec_model, 
 times_model) = model_signal

# Unpack noise and error components
(noise_ra, noise_dec, errors) = error_components

In [5]:
noise_ra_std = np.std(noise_ra)
noise_dec_std = np.std(noise_dec)

# Calculate the overall standard deviation of noise
overall_noise_std = np.sqrt(noise_ra_std**2 + noise_dec_std**2)  # Combine RA and Dec noise in quadrature

# Alternatively, you can calculate the average standard deviation
average_noise_std = (noise_ra_std + noise_dec_std) / 2  # Average of RA and Dec noise standard deviations

In [6]:
print('Overall = ', overall_noise_std)
print('Average = ', average_noise_std)

Overall =  41.48626110132821
Average =  29.307919398407144


In [7]:
signal_ra_obs  = prop_ra_synthetic + parallax_ra_synthetic + planetary_ra_synthetic + noise_ra
signal_dec_obs = prop_dec_synthetic + parallax_dec_synthetic + planetary_dec_synthetic + noise_dec

In [8]:
fitted_params_0P, np_chi_sq = no_planet_fit(inj_params_0P, signal_ra_obs, signal_dec_obs, noise_ra, noise_dec, times_synthetic)

print_parameter_differences(inj_params_0P, fitted_params_0P, return_type='np')

alpha0  : real=177.55505, fitted=177.55791, difference=0.00286
delta0  : real=-52.96075, fitted=-53.17082, difference=0.21007
mu_alpha: real=-134.19021, fitted=-134.18631, difference=0.00390
mu_delta: real=16.28344, fitted=16.27814, difference=0.00530
parallax: real=16.28088, fitted=16.21529, difference=0.06559


In [1]:
noise_ra

NameError: name 'noise_ra' is not defined

In [2]:
noise_dec

NameError: name 'noise_dec' is not defined

In [9]:
fitted_params_1P, wp_chi_sq = one_planet_fit(inj_params_1P, m_star, signal_ra_obs, signal_dec_obs, noise_ra, noise_dec, times_synthetic)

print_parameter_differences(inj_params_1P, fitted_params_1P, return_type='wp')

alpha0         : real=177.55505, fitted=177.55727, difference=0.00222
delta0         : real=-52.96075, fitted=-52.97655, difference=0.01580
mu_alpha       : real=-134.19021, fitted=-134.19090, difference=0.00069
mu_delta       : real=16.28344, fitted=16.28373, difference=0.00028
parallax       : real=16.28088, fitted=16.27503, difference=0.00585
e              : real=0.11100, fitted=0.11214, difference=0.00114
omega          : real=5.47097, fitted=5.36529, difference=0.10568
Omega          : real=1.29885, fitted=1.13336, difference=0.16550
cos_i          : real=0.91861, fitted=0.84760, difference=0.07101
log10(m_planet): real=0.27656, fitted=0.27647, difference=0.00009
log10(P_orb)   : real=-0.16477, fitted=-0.16547, difference=0.00070
t_peri         : real=0.52409, fitted=0.49713, difference=0.02696


In [10]:
detection_result = detection_function(np_chi_sq, wp_chi_sq, inj_params_1P, fitted_params_1P, N_synthetic)

print("np_BIC =", detection_result[0])
print("wp_BIC =", detection_result[1])
print("Delta_BIC =", detection_result[2])
print("detection (yes = 1, no=0) =", detection_result[3])
print("conditions satisfied (Delta BIC>20, log(P) within err, mass within err) =", detection_result[4])

np_BIC = 7.048443332812278
wp_BIC = 42.6615517868888
Delta_BIC = 35.613108454076524
detection (yes = 1, no=0) = 0
conditions satisfied (Delta BIC>20, log(P) within err, mass within err) = [0 1 1]
